In [1]:
import pandas as pd
import pickle
import warnings

warnings.filterwarnings("ignore") 

data = pd.read_csv('test.csv', index_col=0).reset_index(drop=True)
index = 5
test = data.iloc[[index]]

In [21]:
data = {'brand': 'Audi', 'model': 'Q7', 'mileage': 10000, 'yearsInUse': 10, 'damaged': False, 'bodyType': 'внедорожник',
        'fuelType': 'бензин', 'driveType': 'полный', 'transmissionType': 'AT', 'engineCapacity': 3, 'horsePower': 200}

prepared_data = [{
        'brand': data['brand'],
        'years_in_operation': data['yearsInUse'],
        'is_crashed': 1 if data['damaged'] else 0,
        'model': data['brand'] + ' ' + data['model'],
        'mileage': data['mileage'],
        'body_type': data['bodyType'],
        'drive_type': data['driveType'],
        'fuel_type': data['fuelType'],
        'engine_volume': data['engineCapacity'],
        'transmission_type': data['transmissionType'],
        'power': data['horsePower'],
    }]
test = pd.DataFrame(prepared_data)

In [22]:
def translate_data(car):
    rename_dict = pickle.load(open('../utils/rename_dict.pkl', 'rb'))
    car['drive_type'] = car['drive_type'].apply(lambda x: rename_dict[x])
    car['fuel_type'] = car['fuel_type'].apply(lambda x: rename_dict[x])

In [23]:
def one_hot_encoding(car):
    one_hot_cols = ['drive_type', 'fuel_type', 'transmission_type']
    result = car.drop(columns=one_hot_cols)

    for col in one_hot_cols:
        encoder = pickle.load(
            open(f'../utils/{col}_one_hot.pkl', 'rb'))
        
        col_encoded = pd.DataFrame(encoder.transform(car[[col]]).toarray(
        ), columns=encoder.get_feature_names_out([col]))

        result = pd.concat((result, col_encoded), axis=1)

    return result

In [24]:
def target_encoding(car):
    target_encoded_cols = ['body_type', 'brand', 'model']

    for col in target_encoded_cols:
        encoder = pickle.load(
            open(f'../utils/{col}_target.pkl', 'rb'))
        
        car[col] = encoder[car[col].values[0]]

In [25]:
def prepare_data(car):
    #car.drop(columns='price', inplace=True)
    car.reset_index(drop=True, inplace=True)
    translate_data(car)
    target_encoding(car)
    return one_hot_encoding(car)

In [ ]:
scaler = pickle.load(open('../utils/scaler.pkl', 'rb'))
model = pickle.load(open('../utils/catboost_model.pkl', 'rb'))

ready_data = prepare_data(test)

test_scaled = scaler.transform(ready_data)

Предсказание модели - 7214599


In [27]:
print(f'Предсказание модели - {int(model.predict(test_scaled)[0])}')

Предсказание модели - 7214599
